In [2]:
# 数据提取

import os
import json
import random
# 设置随机种子
random.seed(42)
root='/media/jh/新加卷1/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/'

def process_data(root,dev,en_de):
    datas=[]
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'subject' in d:
                tmp=d
                break
        item={}
        item['title']=tmp['title']
        item['abstract']=tmp['abstract']
        item['dcterms:subject']=tmp['dcterms:subject']
        item['subject']=tmp['subject']
        datas.append(item)
    for dirs in os.listdir(root+f'{all_core}/data/{dev}'):
        for file in os.listdir(root+f'{all_core}/data/{dev}/{dirs}/{en_de}'):
            get_train_data(root+f'{all_core}/data/{dev}/{dirs}/{en_de}/{file}')
    return datas

all_core='tib-core-subjects'
en_de='en'
dev='train'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))
en_de='de'
dev='train'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))

en_de='en'
dev='dev'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))
en_de='de'
dev='dev'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))

all_core='all-subjects'
en_de='en'
dev='train'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))
en_de='de'
dev='train'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))

en_de='en'
dev='dev'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))
en_de='de'
dev='dev'
datas=process_data(root,dev,en_de)
with open(f'test/{all_core}_{dev}_{en_de}.json','w') as f:
    f.write(json.dumps(datas,ensure_ascii=False, indent=4))

In [ ]:
# 混合负样本的单标签样本数据集制作
import json
import random
#设置随机种子
random.seed(42)
root='/media/jh/新加卷1'
file_path=f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json' #GND-Subjects-tib-core.json'
with open(file_path,'r') as f:
    data=json.loads(f.read())
gnd_id2name={}
all_codes=[]
for i in data:
    all_codes.append(i['Code'])
    gnd_id2name[i['Code']]=i['Name']
    for j in i['Alternate Name']:
        gnd_id2name[i['Code']]=gnd_id2name[i['Code']]+','+j


def get_train_data(datas,x_train):
    cnt=0
    for t in datas:
        if type(t['title'])!=str:
            t['title']=' '.join(t['title'])
        if type(t['abstract'])!=str:
            t['abstract']=' '.join(t['abstract'])
        # s1s.append(t['title']+'.'+t['abstract'])
        codes=[]
        for gnd in t['dcterms:subject']:
            if type(gnd)==dict:
                gnd_id=gnd['@id']
                codes.append(gnd_id)
            else:
                continue
        for gnd in t['dcterms:subject']:
            if type(gnd)==dict:
                gnd_id=gnd['@id']
            else:
                continue
            cnt+=1
            if gnd_id in gnd_id2name:
                gnd_set.add(gnd_id)
                gnd_val=gnd_id2name[gnd_id]
                item={}
                item['sentence2']= 'Name is '+gnd_val
                item['sentence1']=t['title']+'.'+t['abstract']
                item['label']=1
                x_train.append(item)
                # 随机取出属于all_codes但不属于codes的gnd_id
                while True:
                    gnd_id=random.choice(all_codes)
                    if gnd_id not in codes:
                        break
                gnd_val=gnd_id2name[gnd_id]
                item={}
                item['sentence2']= 'Name is '+gnd_val
                item['sentence1']=t['title']+'.'+t['abstract']
                item['label']=0
                x_train.append(item)
gnd_set=set()
x_train=[]
datas=[]
# 由process_data函数提取并制作数据集all-subjects_train_de.json等
all_core='all-subjects'
with open(f'test/{all_core}_train_de.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_train)
datas=[]
with open(f'test/{all_core}_train_en.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_train)
print('len(x_train)=',len(x_train))

x_dev=[]
datas=[]
with open(f'test/{all_core}_dev_de.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_dev)
datas=[]
with open(f'test/{all_core}_dev_en.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_dev)

print('len(x_dev)=',len(x_dev))
print('len(gnd_set/all_codes_used)=',len(gnd_set))
print('len(all_codes)=',len(all_codes))

In [ ]:
# 混合负样本的多标签样本数据集制作
import json
import random
#设置随机种子
random.seed(42)
root='/media/jh/新加卷1'
file_path=f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-tib-core.json' #GND-Subjects-tib-core.json'
with open(file_path,'r') as f:
    data=json.loads(f.read())
core_codes=[]
for i in data:
    core_codes.append(i['Code'])
root='/media/jh/新加卷1'
file_path=f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json' #GND-Subjects-tib-core.json'
with open(file_path,'r') as f:
    data=json.loads(f.read())
gnd_id2name={}
all_codes=[]
for i in data:
    all_codes.append(i['Code'])
    gnd_id2name[i['Code']]=i['Name']
    for j in i['Alternate Name']:
        gnd_id2name[i['Code']]=gnd_id2name[i['Code']]+','+j


def get_train_data(datas,x_train):
    cnt=0
    for t in datas:
        if type(t['title'])!=str:
            t['title']=' '.join(t['title'])
        if type(t['abstract'])!=str:
            t['abstract']=' '.join(t['abstract'])
        # s1s.append(t['title']+'.'+t['abstract'])
        codes=[]
        for gnd in t['dcterms:subject']:
            if type(gnd)==dict:
                gnd_id=gnd['@id']
                codes.append(gnd_id)
            else:
                continue
        item1={}
        item1['sentence1']=t['title']+'.'+t['abstract']
        item1['label']=1
        item1['sentence2']='These are the names of the subjects:'
        item2={}
        item2['sentence1']=t['title']+'.'+t['abstract']
        item2['label']=0
        item2['sentence2']='These are the names of the subjects:'
        for gnd in t['dcterms:subject']:
            if type(gnd)==dict:
                gnd_id=gnd['@id']
            else:
                continue
            cnt+=1
            if gnd_id in gnd_id2name:
                gnd_set.add(gnd_id)
                gnd_val=gnd_id2name[gnd_id]
                item1['sentence2'] += gnd_val+','
                # 随机取出属于all_codes但不属于codes的gnd_id
                while True:
                    gnd_id=random.choice(all_codes)
                    if gnd_id not in codes:
                        break
                gnd_val=gnd_id2name[gnd_id]
                item2['sentence2'] += gnd_val+','
        x_train.append(item1)
        x_train.append(item2)
gnd_set=set()
x_train=[]
datas=[]
# 由process_data函数提取并制作数据集all-subjects_train_de.json等
all_core='tib-core-subjects'
with open(f'test/{all_core}_train_de.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_train)
datas=[]
with open(f'test/{all_core}_train_en.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_train)
print('len(x_train)=',len(x_train))

x_dev=[]
datas=[]
with open(f'test/{all_core}_dev_de.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_dev)
datas=[]
with open(f'test/{all_core}_dev_en.json','r') as f:
    datas+=json.loads(f.read())
get_train_data(datas,x_dev)

print('len(x_dev)=',len(x_dev))
print('len(gnd_set/core_codes_used)=',len(gnd_set))
print('len(core_codes)=',len(core_codes))
print('len(all_codes)=',len(all_codes))

In [ ]:
# 使用上述数据及制作方法的一种进行训练。 把wandb API_KEY换成自己的
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from datasets import Dataset
import os
os.environ["WANDB_API_KEY"] = "XXX"
# os.environ["WANDB_MODE"] = "offline"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

model = SentenceTransformer("sentence-transformers/all-distilroberta-v1")

random.shuffle(x_train)
random.shuffle(x_dev)

train_dataset = Dataset.from_dict({
    "sentence1": [item['sentence1'] for item in x_train],
    "sentence2": [item['sentence2'] for item in x_train],
    "label": [item['label'] for item in x_train],
})
eval_dataset = Dataset.from_dict({
    "sentence1": [item['sentence1'] for item in x_dev],
    "sentence2": [item['sentence2'] for item in x_dev],
    "label": [item['label'] for item in x_dev],
})
loss = losses.ContrastiveLoss(model)
# 根据模型和数据集更改下面参数
args=SentenceTransformerTrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, #根据数据集大小更改
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="no",
    save_total_limit=0,
)
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
)
trainer.train()

save_path="out_models/all-distilroberta-v1"
if not os.path.exists(save_path):
    os.makedirs(save_path)
model.save(save_path)